# METADATA CONVERSION

Download dataset IMDB - faces only (7 GB) from https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/
Copy the downloaded directory imdb_crop in the project folder dataset/, then run this notebook.

This notebook converts the imdb_crop images metadata from Matlab format into Pandas format, so they can be used with Python.

The ML models for classification must inherit from src/models/Model, the DataManager class reads the images and extracts the labels

In [2]:
import numpy as np
import scipy.io as sio
import cv2
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm

In [3]:
# Matlab file path
METADATA_CROPPED_FILE_MATLAB = '../dataset/imdb_crop/imdb.mat'

In [3]:
mat_obj = sio.loadmat(METADATA_CROPPED_FILE_MATLAB)
mat_dta = mat_obj['imdb']
data_dict = {n: mat_dta[n][0, 0][0] for n in mat_dta.dtype.names}
# Extract celebrities names
celeb_names_dict = {k: v for k, v in enumerate(data_dict['celeb_names'], 1)}
# Remove key from dict
_ = data_dict.pop('celeb_names', None)

In [4]:
df = pd.DataFrame.from_dict(data_dict)
df.head()

,dob,photo_taken,full_path,gender,name,face_location,face_score,second_face_score,celeb_id
0,693726,1968,[01/nm0000001_rm124825600_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[1072.926, 161.838, 1214.7839999999999, 303.6...",1.459693,1.118973,6488
1,693726,1970,[01/nm0000001_rm3343756032_1899-5-10_1970.jpg],1.0,[Fred Astaire],"[[477.184, 100.352, 622.592, 245.76]]",2.543198,1.852008,6488
2,693726,1968,[01/nm0000001_rm577153792_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[114.96964308962852, 114.96964308962852, 451....",3.455579,2.985660,6488
3,693726,1968,[01/nm0000001_rm946909184_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[622.8855056426588, 424.21750383700805, 844.3...",1.872117,NaN,6488
4,693726,1968,[01/nm0000001_rm980463616_1899-5-10_1968.jpg],1.0,[Fred Astaire],"[[1013.8590023603723, 233.8820422075853, 1201....",1.158766,NaN,6488


In [5]:
from datetime import datetime, timedelta
matlab2datetime = lambda matlab_datenum: (datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1)).year

# Extract full path
df['full_path'] = df['full_path'].apply(lambda x: x[0]).astype('str')
# Extract face location
df['face_location'] = df['face_location'].apply(lambda x: x[0])
# Extract face location
df['name'] = df['name'].apply(lambda x: x[0]).astype('str')
# Convert dab to standard datetime
df['standard_dob'] = df['dob'].apply(matlab2datetime).astype('uint16')
# Compute age
df['age'] = df.apply(lambda x:x['photo_taken']-x['standard_dob'], axis=1).astype('uint16')

In [6]:
# Reorder columns
columns = df.columns.tolist()
head = ['age', 'gender', 'full_path', 'name', 'face_location']
tail = set(columns) - set(head)
ordered_columns = head + list(tail)
# Ordered columns
df = df[ordered_columns]
df.head()

,age,gender,full_path,name,face_location,second_face_score,standard_dob,celeb_id,face_score,photo_taken,dob
0,68,1.0,01/nm0000001_rm124825600_1899-5-10_1968.jpg,Fred Astaire,"[1072.926, 161.838, 1214.7839999999999, 303.69...",1.118973,1900,6488,1.459693,1968,693726
1,70,1.0,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,Fred Astaire,"[477.184, 100.352, 622.592, 245.76]",1.852008,1900,6488,2.543198,1970,693726
2,68,1.0,01/nm0000001_rm577153792_1899-5-10_1968.jpg,Fred Astaire,"[114.96964308962852, 114.96964308962852, 451.6...",2.985660,1900,6488,3.455579,1968,693726
3,68,1.0,01/nm0000001_rm946909184_1899-5-10_1968.jpg,Fred Astaire,"[622.8855056426588, 424.21750383700805, 844.33...",NaN,1900,6488,1.872117,1968,693726
4,68,1.0,01/nm0000001_rm980463616_1899-5-10_1968.jpg,Fred Astaire,"[1013.8590023603723, 233.8820422075853, 1201.5...",NaN,1900,6488,1.158766,1968,693726


In [7]:
# Remove invalid rows
len_before = len(df)
print('Len before: ', len_before)
df = df.query('age<=100')
df = df[df.gender.notna()]
df = df[df.age.notna()]
len_after = len(df)
print('Lean after: ', len_after)
print(f'Invalid rows: {(1 - len_after/len_before)*100:.3f}%')

Len before:  460723
Lean after:  451571
Invalid rows: 1.986%


In [8]:
# CHECK FUNCTIONS: if the image is too little then we discard it; if the image has padding, we discard it

def areRowsEqual(rows):
    check = False
    for i in range(1, len(rows)):
        if (rows[i-1] == rows[i]).all():
            check = True
        else:
            check = False
    return check

def areColsEqual(cols):
    return areRowsEqual(np.moveaxis(cols, 0, 1))

def isImagePadded(img, number_equal_rows=5):
    nr = number_equal_rows
    return areRowsEqual(img[:nr,:,:]) or areRowsEqual(img[-nr:,:,:]) or \
            areColsEqual(img[:,:nr,:]) or areColsEqual(img[:,-nr:,:])

def isImageTooLittle(img, smallest_dim=124):
    if img.shape[0] <= smallest_dim or img.shape[1] <= smallest_dim:
        return True
    else:
        return False

In [9]:
# Remove invalid images
len_before = len(df)
print('Len before: ', len_before)

with tqdm(total=df.shape[0]) as pbar:
    for index,row in df.iterrows():
        img = cv2.imread('../dataset/imdb_crop/' + row.full_path)
        if isImageTooLittle(img) or isImagePadded(img):
            df.drop(index, inplace=True)
        pbar.update(1)

len_after = len(df)
print('Lean after: ', len_after)
print(f'Invalid rows: {(1 - len_after/len_before)*100:.3f}%')

Len before:  451571


100%|██████████| 451571/451571 [3:10:40<00:00, 39.47it/s]   

Lean after:  451571
Invalid rows: 0.000%


In [13]:
print('Lean after: ', len_after)
print(f'Invalid rows: {(1 - len_after/len_before)*100:.3f}%')

Lean after:  280844
Invalid rows: 37.807%


In [4]:
from os import path

# Dump dataframe
head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)

path_serialized = path.join(head, 'imdb.pickle')
df.to_pickle(path_serialized)

NameError: name 'df' is not defined

In [5]:
# Test read
df = pd.read_pickle(path_serialized)
df.head()

,age,gender,full_path,name,face_location,second_face_score,standard_dob,celeb_id,face_score,photo_taken,dob
0,68,1.0,01/nm0000001_rm124825600_1899-5-10_1968.jpg,Fred Astaire,"[1072.926, 161.838, 1214.7839999999999, 303.69...",1.118973,1900,6488,1.459693,1968,693726
1,70,1.0,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,Fred Astaire,"[477.184, 100.352, 622.592, 245.76]",1.852008,1900,6488,2.543198,1970,693726
3,68,1.0,01/nm0000001_rm946909184_1899-5-10_1968.jpg,Fred Astaire,"[622.8855056426588, 424.21750383700805, 844.33...",NaN,1900,6488,1.872117,1968,693726
4,68,1.0,01/nm0000001_rm980463616_1899-5-10_1968.jpg,Fred Astaire,"[1013.8590023603723, 233.8820422075853, 1201.5...",NaN,1900,6488,1.158766,1968,693726
5,66,0.0,02/nm0000002_rm1075631616_1924-9-16_1991.jpg,Lauren Bacall,"[686.6768205940973, 458.4672137293982, 1198.10...",1.345335,1925,11516,3.490563,1991,702986


In [6]:
df.describe()

,age,gender,second_face_score,standard_dob,celeb_id,face_score,photo_taken,dob
count,280844.000000,280844.000000,146906.000000,280844.000000,280844.000000,280844.000000,280844.000000,280844.000000
mean,36.164258,0.577128,2.485057,1969.365993,10119.464236,3.362345,2005.530252,719113.598813
std,13.148274,0.494016,1.092884,15.691095,5739.357341,1.238619,9.915925,5730.534448
min,0.000000,0.000000,0.730926,1884.000000,1.000000,0.730919,1961.000000,687776.000000
25%,27.000000,0.000000,1.593553,1962.000000,5355.000000,2.435448,2004.000000,716241.000000
50%,35.000000,1.000000,2.373423,1972.000000,10068.000000,3.382505,2009.000000,719928.000000
75%,44.000000,1.000000,3.277556,1980.000000,14925.000000,4.313906,2012.000000,723127.000000
max,100.000000,1.000000,6.395435,2009.000000,20284.000000,7.381689,2015.000000,733668.000000
